In [74]:
# Copyright 2019 Eugene Maslovich
# ehpc@ehpc.io

In [163]:
import time
from mpmath import mp
import mpmath.libmp
mpmath.libmp.BACKEND

'gmpy'

In [187]:
# Проблему замощения прямоугольника AxB прямоугольником 1x2 можно рассматривать как классическую проблему замощения плоскости,
# которая аналитически решается раскрашивание плоскости шахматной доской. Можно былло копать и дальше в этом направлении, но
# если представить прямоугольник в виде графа, вершины которого - центры раскрашенных квадратов, то проблема сводится к
# нахождению числа совершенных сочетаний планарного графа, т.к. прямоугольник 1x2 является по сути ребром такого графа.
# Существуюет алгоритм, позволяющий вычислить общую задачу нахождения совершенных сочетаний за полиномиальное время - 
# FKT (Fisher, Kasteleyn, Temperley). В случае нашей задачи, всё сводится к формуле https://books.google.nl/books?id=4pCKDwAAQBAJ&lpg=PA145&ots=8pU1IQa6Ts&dq=Temperley%20Fisher%20Kasteleyn&hl=ru&pg=PA157#v=onepage&q=Temperley%20Fisher%20Kasteleyn&f=false.
# Жуткие синусы и косинусы получаются из вывода детерминанта матрицы алгоритма FKT и описаны в работах Фишера и Кастелейна.
# Для преодоления барьера точности используем модуль mpmath.
# Проверить корректность функции можно сравнением результатов с последовательностью http://oeis.org/A004003.
def pack(a, b):
    if a % 2 != 0 and b %2 != 0:
        return 0
    else:
        p = None
        jtop = math.ceil(a / 2) + 1
        ktop = math.ceil(b / 2) + 1
        ap1 = mp.fadd(a, 1)
        bp1 = mp.fadd(b, 1)
        for j in range(1, jtop):
            for k in range(1, ktop):
                f1cos = mp.cos(mp.fdiv(mp.fmul(mp.pi, mp.mpf(j)), ap1))
                f1 = mp.fmul(4, mp.power(f1cos, 2))
                f2cos = mp.cos(mp.fdiv(mp.fmul(mp.pi, mp.mpf(k)), bp1))
                f2 = mp.fmul(4, mp.power(f2cos, 2))
                f = mp.fadd(f1, f2)
                if p is None:
                    p = f
                else:
                    p = mp.fmul(p, f)
        if p is None:
            return '0'
        else:
            val = mp.nstr(mp.nint(p), 10000)
            # Возможна потеря точности, если вернулся False
            if len(val) > mp.dps:
                return [val.split('.')[0], False, len(val)]
            else:
                return [val.split('.')[0], True, len(val)]

In [188]:
# Здесь мы на коленке проверяем корректность алгоритма на известных значениях
def test():
    mp.dps = 100
    n = 10
    # Числа для проверки алгоритма http://oeis.org/A004003
    tests = ['2', '36', '6728', '12988816', '258584046368', '53060477521960000', '112202208776036178000000',
             '2444888770250892795802079170816', '548943583215388338077567813208427340288',
             '1269984011256235834242602753102293934298576249856']
    for a in range(1, n + 1):
        val = pack(2 * a, 2 * a)[0]
        print("pack{}({}, {}) = ".format(a, 2 * a, 2 * a), val, 'WRONG' if val != tests[a - 1] else 'OK')
test()
print("Precision: ".format(), mp)

pack1(2, 2) =  2 OK
pack2(4, 4) =  36 OK
pack3(6, 6) =  6728 OK
pack4(8, 8) =  12988816 OK
pack5(10, 10) =  258584046368 OK
pack6(12, 12) =  53060477521960000 OK
pack7(14, 14) =  112202208776036178000000 OK
pack8(16, 16) =  2444888770250892795802079170816 OK
pack9(18, 18) =  548943583215388338077567813208427340288 OK
pack10(20, 20) =  1269984011256235834242602753102293934298576249856 OK
Precision:  Mpmath settings:
  mp.prec = 336               [default: 53]
  mp.dps = 100                [default: 15]
  mp.trap_complex = False     [default: False]


In [190]:
%%time
# Эта функция уже пытается вычислять все возможные значения и останавливается только тогда,
# когда превышен указанный порог скорости вычисления.
def calcmax(max_time):
    mp.dps = 100
    val = None
    a = 1
    time_total = 0.1
    while True:
        for b in range(a, 2 * a + 1, 2 if time_total < max_time / 2 else 1):
            start = time.time()
            val = pack(a, b)
            time_total = time.time() - start
            if time_total > max_time:
                print("Time (s): {}. pack({}, {}) = {}. Possibly incorrect: {}".format(time_total, a, b, val[0], "false" if val[1] else "true"))
                return max
        if time_total < max_time / 2:
            a = a * 2
        else:
            a = a + 1
calcmax(0.01)

Time (s): 0.01000070571899414. pack(16, 30) = 7497867331813387563841578587730180239918872416374766607861. Possibly incorrect: false
Wall time: 74.8 ms


<function max>

In [199]:
# Функция вывода вычисления
def calc_pretty(a, b):
    val = pack(a, b)
    print("pack({}, {}) = {}. Possibly incorrect: {}. Digits: {}.".format(a, b, val[0], "false" if val[1] else "true", val[2]))

In [202]:
%%time
mp.dps = 100
calc_pretty(20, 30)

pack(20, 30) = 6217539537964764012601603771542270056953148716152633313878015867124216625. Possibly incorrect: false. Digits: 75.
Wall time: 11 ms


In [204]:
%%time
mp.dps = 2600
calc_pretty(100, 200)

pack(100, 200) = 13184626568252634672850731534369811480035905349663430100982867941140244181342264636212655191058561972818541083395282092787408266067300860524393108721683793545658199102881727357810488930922381918388314128962566699974841113909674378795326966514390085003755502456903073461750529783950391581170209720097316999674839569771093292285529258035999881785318472693384129728244460758715310701060129341244940022622910758667782220078172962770560419228321069896219350502245502614484070894433011171402191171663762697489973334320489969492161323989132854205885326859545536642429515573742532642153773454502116226784822216529312729830223876858114462478008660828669902726402630689380371883529751793493061784469063036543056240058010574591336857866686195407946047020090762279955090850954747680133988697334273785370681840116926747137251427570097280897042067637471675132002385656327778720524637464311897537966998990539016844620824480873967215837728914764297370400718749638166315806611885002074904999830736023

In [206]:
%%time
mp.dps = 8000
calc_pretty(200, 300)

pack(200, 300) = 11854917398922217249204487018447392756115184341015836625593051732287263513585613894836575367272948760319473624699590890317511608889344762832363336747362873456321377938210357960170554933290287867344306668388919251433791578282995987670841151686045934740708927829305291847389416314362974581176090116192159028979184294817907261955067615117094738922767333209040515350979464167760596523061757512914575867890490143041389122855952183550564211497161859774438621978906986377768712684413927099796242997790003529174358403966371025995154595081852140327847552310658861258204910502032893046085888430714127857099190239668384434266277570348255563354986624077758465570093741215244243099893751387699620776070284286393407087992180267874351237625023840291422626346508089471154036918059349336974860100095882410942412119904930313943953717731502495074170003970607673128493077287150130695629198259460746411674936972319460248709627615751542998787283026502776449108371448973627868754248447008866101279298292963